In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from random import gauss
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats as stats

%matplotlib inline
pd.set_option('display.max_columns', None) #shows all columns on dataframe
pd.set_option('display.float_format', lambda x: '%.2f' % x) #applied to every df in this notebook

In [2]:
df = pd.read_csv('./data/kc_house_data.csv')

In [3]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.00,3,1.00,1180,5650,1.00,NaN,NONE,Average,7 Average,1180,0.0,1955,0.00,98178,47.51,-122.26,1340,5650
1,6414100192,12/9/2014,538000.00,3,2.25,2570,7242,2.00,NO,NONE,Average,7 Average,2170,400.0,1951,1991.00,98125,47.72,-122.32,1690,7639
2,5631500400,2/25/2015,180000.00,2,1.00,770,10000,1.00,NO,NONE,Average,6 Low Average,770,0.0,1933,nan,98028,47.74,-122.23,2720,8062
3,2487200875,12/9/2014,604000.00,4,3.00,1960,5000,1.00,NO,NONE,Very Good,7 Average,1050,910.0,1965,0.00,98136,47.52,-122.39,1360,5000
4,1954400510,2/18/2015,510000.00,3,2.00,1680,8080,1.00,NO,NONE,Average,8 Good,1680,0.0,1987,0.00,98074,47.62,-122.05,1800,7503


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [5]:
df['condition'].value_counts()
#difficult to make reccomendation base off of this since there's a lot of ways to define these terms
#use grade which is a little less vague

Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

In [6]:
df['yr_renovated'].value_counts()
#dropping columns base on the number of zeros
#potentially compared the zeros to the those that have ben renovated to see if there is a significant difference

0.00       17011
2014.00       73
2003.00       31
2013.00       31
2007.00       30
           ...  
1946.00        1
1959.00        1
1971.00        1
1951.00        1
1954.00        1
Name: yr_renovated, Length: 70, dtype: int64

In [45]:
#think that view might be important so we drop the nulls there for now
df_copy = df.copy()
df_copy['view'] = df_copy['view'].fillna(value = 'NO RECORD')

In [46]:
df_copy = df_copy.sort_values(by=['id', 'date'])
df_copy = df_copy.drop_duplicates(subset=['id'], keep='last')

In [47]:
numbers = df_copy.select_dtypes(np.number)

In [48]:
df_2 = numbers.drop(['id', 'lat', 'long', 'zipcode'], axis = 1)

In [49]:
#picture = sns.pairplot(numbers)

In [50]:
#fig, ax = plt.subplots(figsize=(10,10))
#heat = sns.heatmap(df_2.corr(), annot = True)
#plt.savefig('heat.png',bbox = 'tight')

In [51]:
#visually the price of average, fair and good look the about the same so we would need to run a significant test to see
#excellent and none looks to be potentially significant
#use of anova here to see and ad-hoc tests

In [52]:
df_copy.yr_renovated.fillna(value = 0, inplace = True)
#fill them with zero, might be mention in the next step to look at
#for now we can drop 

In [53]:
#we have some of the houses appearing more than once so we are trying to drop
#trial = copy_df.groupby(['id']).date.transform(max)
#trial_cleaned = copy_df[copy_df.id == trial]

In [54]:
#making the categorical variable
cate_column = df_copy[['grade', 'view']]

df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21420 entries, 2494 to 15937
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21420 non-null  int64  
 1   date           21420 non-null  object 
 2   price          21420 non-null  float64
 3   bedrooms       21420 non-null  int64  
 4   bathrooms      21420 non-null  float64
 5   sqft_living    21420 non-null  int64  
 6   sqft_lot       21420 non-null  int64  
 7   floors         21420 non-null  float64
 8   waterfront     19070 non-null  object 
 9   view           21420 non-null  object 
 10  condition      21420 non-null  object 
 11  grade          21420 non-null  object 
 12  sqft_above     21420 non-null  int64  
 13  sqft_basement  21420 non-null  object 
 14  yr_built       21420 non-null  int64  
 15  yr_renovated   21420 non-null  float64
 16  zipcode        21420 non-null  int64  
 17  lat            21420 non-null  float64
 18  lon

In [55]:
ohe = OneHotEncoder(drop='first')

column_name = ['grade', 'view']
features = cate_column.copy()

features2 = features[column_name]

cate_ohe = ohe.fit_transform(features2)

cate_ohe

<21420x15 sparse matrix of type '<class 'numpy.float64'>'
	with 40754 stored elements in Compressed Sparse Row format>

In [56]:
name = ohe.get_feature_names()

In [57]:
cate_parsed = pd.DataFrame(cate_ohe.todense(), columns = name)

In [58]:
cate_parsed
#X0 = grade
#x1 = view

,x0_11 Excellent,x0_12 Luxury,x0_13 Mansion,x0_3 Poor,x0_4 Low,x0_5 Fair,x0_6 Low Average,x0_7 Average,x0_8 Good,x0_9 Better,x1_EXCELLENT,x1_FAIR,x1_GOOD,x1_NO RECORD,x1_NONE
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
3,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21415,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
21416,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
21417,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
21418,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00


In [59]:
#to potentially drop sqft_lot and sqft_lot15
ss = StandardScaler()
#scaled_ = ss.fit_transform(df_2)

columns_name1 = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'floors', 'sqft_above']
numbers_scaling = df_2.copy().drop(['yr_renovated','sqft_lot15', 'sqft_lot', 'yr_built'], axis = 1)

features= numbers_scaling[columns_name1]

scale1 = ss.fit(features)
scale2 = scale1.transform(features)
numbers_scaling[columns_name1] = scale2

numbers_scaling

,price,bedrooms,bathrooms,sqft_living,floors,sqft_above,sqft_living15
2494,-0.71,2.84,1.15,0.34,0.93,0.73,2060
6729,0.29,0.68,-0.48,-0.03,-0.92,-0.76,2590
8404,-0.38,-0.40,-1.45,-0.68,-0.92,-0.40,2250
8800,-0.83,-0.40,-1.45,-0.71,0.01,-0.94,1290
3553,-0.38,0.68,-0.15,-0.47,-0.92,-1.24,1480
...,...,...,...,...,...,...,...
16723,-0.48,1.76,-0.15,-0.53,0.01,-0.23,1190
3257,-0.44,-1.48,-1.45,-1.14,-0.92,-1.15,1930
7614,-0.55,-0.40,-1.45,-1.07,-0.92,-1.29,1510
20963,-0.38,-1.48,-0.48,-0.73,0.01,-1.08,1440


In [60]:
scaled_and_parse_df = pd.concat([numbers_scaling, cate_parsed], axis = 1)

In [61]:
numbers_scaling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21420 entries, 2494 to 15937
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21420 non-null  float64
 1   bedrooms       21420 non-null  float64
 2   bathrooms      21420 non-null  float64
 3   sqft_living    21420 non-null  float64
 4   floors         21420 non-null  float64
 5   sqft_above     21420 non-null  float64
 6   sqft_living15  21420 non-null  int64  
dtypes: float64(6), int64(1)
memory usage: 1.9 MB


In [62]:
cate_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21420 entries, 0 to 21419
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   x0_11 Excellent   21420 non-null  float64
 1   x0_12 Luxury      21420 non-null  float64
 2   x0_13 Mansion     21420 non-null  float64
 3   x0_3 Poor         21420 non-null  float64
 4   x0_4 Low          21420 non-null  float64
 5   x0_5 Fair         21420 non-null  float64
 6   x0_6 Low Average  21420 non-null  float64
 7   x0_7 Average      21420 non-null  float64
 8   x0_8 Good         21420 non-null  float64
 9   x0_9 Better       21420 non-null  float64
 10  x1_EXCELLENT      21420 non-null  float64
 11  x1_FAIR           21420 non-null  float64
 12  x1_GOOD           21420 non-null  float64
 13  x1_NO RECORD      21420 non-null  float64
 14  x1_NONE           21420 non-null  float64
dtypes: float64(15)
memory usage: 2.5 MB


In [63]:
#for our regression and simple linear model
scaled_and_parse_df

,price,bedrooms,bathrooms,sqft_living,floors,sqft_above,sqft_living15,x0_11 Excellent,x0_12 Luxury,x0_13 Mansion,x0_3 Poor,x0_4 Low,x0_5 Fair,x0_6 Low Average,x0_7 Average,x0_8 Good,x0_9 Better,x1_EXCELLENT,x1_FAIR,x1_GOOD,x1_NO RECORD,x1_NONE
0,-0.87,-0.40,-1.45,-0.98,-0.92,-0.74,1340.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
1,-0.01,-0.40,0.17,0.53,0.93,0.46,1690.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00
2,-0.98,-1.48,-1.45,-1.43,-0.92,-1.23,2720.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
3,0.17,0.68,1.15,-0.13,-0.92,-0.89,1360.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
4,-0.08,-0.40,-0.15,-0.44,-0.92,-0.13,1800.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,-0.49,-0.40,0.50,-0.60,2.78,-0.32,1530.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
21593,-0.38,0.68,0.50,0.25,0.93,0.63,1830.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
21594,-0.38,-1.48,-1.78,-1.16,0.93,-0.93,1020.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
21595,-0.38,-0.40,0.50,-0.53,0.93,-0.23,1410.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [64]:
scaled_and_parse_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21596 entries, 0 to 21596
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             21420 non-null  float64
 1   bedrooms          21420 non-null  float64
 2   bathrooms         21420 non-null  float64
 3   sqft_living       21420 non-null  float64
 4   floors            21420 non-null  float64
 5   sqft_above        21420 non-null  float64
 6   sqft_living15     21420 non-null  float64
 7   x0_11 Excellent   21420 non-null  float64
 8   x0_12 Luxury      21420 non-null  float64
 9   x0_13 Mansion     21420 non-null  float64
 10  x0_3 Poor         21420 non-null  float64
 11  x0_4 Low          21420 non-null  float64
 12  x0_5 Fair         21420 non-null  float64
 13  x0_6 Low Average  21420 non-null  float64
 14  x0_7 Average      21420 non-null  float64
 15  x0_8 Good         21420 non-null  float64
 16  x0_9 Better       21420 non-null  float6

In [65]:
#check for nulls, so clean to work with for regression
scaled_and_parse_df.isnull().sum().sum()

3872